# Hoàng Phương Linh - 18020758

In [ ]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
import sklearn.ensemble as ens
import xgboost as xgb
import json
import os
import re
from sklearn import preprocessing
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms as T
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from keras.applications import VGG19
from keras.applications import DenseNet121
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Input
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import lightgbm as lgb

# Load Sentiment Data từ file Json

In [ ]:
def SentimentData(data):
    sentiment = [0,0,0,0,0,0]
    if data['PetID']+'.json' in os.listdir('../input/petfinder-adoption-prediction/train_sentiment'):
        f = open('../input/petfinder-adoption-prediction/train_sentiment/'+data['PetID']+'.json', 'r')
        jsonContent = json.loads(f.read())
        # Lẩy magnitude và score của từng câu:
        mag = np.array([i['sentiment']['magnitude'] for i in jsonContent['sentences']])
        sco = np.array([i['sentiment']['score'] for i in jsonContent['sentences']])
        # Trả về magnitude và score của cả đoạn và trung bình,tổng của magnitude,score với từng câu:
        sentiment = [np.sum(mag),np.sum(sco),np.mean(mag),np.mean(sco),
                     jsonContent['documentSentiment']['magnitude'],jsonContent['documentSentiment']['score']]
        print(data['PetID'],end='\r')
    return sentiment

def SentimentDataEntity(data):
    sentiment = ''
    if data['PetID']+'.json' in os.listdir('../input/petfinder-adoption-prediction/train_sentiment'):
        f = open('../input/petfinder-adoption-prediction/train_sentiment/'+data['PetID']+'.json', 'r')
        jsonContent = json.loads(f.read())
        # Trả về 1 câu nối name của các đối tuợng entities của đoạn Description:
        sentiment = ' '.join([i['name'] for i in jsonContent['entities']])
        print(data['PetID'],end='\r')
    return sentiment

* **Vấn đề:** load dữ liệu từ file json Sentiment miêu tả cảm xúc nhận được từ Description của post

* **Cách làm:** tìm các PetID có file json trong thư mục bằng thư viện os. Load file json thành dict, lấy các trường
    *  magnitude của cả đoạn văn (chỉ độ lớn cảm xúc từ 0 -> )
    *  score của cả đoạn văn (chỉ độ tích cực cảm xúc từ -1 -> 1)
    *  trung bình magnitude của từng câu
    *  tổng magnitude của từng câu
    *  trung bình score của từng câu
    *  tổng score của từng câu
    *  name của các đối tượng trong câu (dog, cat, house,...)
* Những PetID không có file sentiment thì để [0,0,0,0,0,0], [''], hoặc thiếu thì để chỗ đấy = 0
* **Kết quả:**
    * mảng 14993 hàng chứa các thuộc tính trên cho tập train
    * mảng 3971 hàng chứa các thuộc tính trên cho tập test
    
   

# Load Metadata từ file Json

In [ ]:
def Metadata(data):
    metadata=[0,'',0,0]
    if data['PetID']+'-1.json' in os.listdir('../input/petfinder-adoption-prediction/train_metadata'):
        f = open('../input/petfinder-adoption-prediction/train_metadata/'+data['PetID']+'-1.json', 'r')
        jsonContent = json.loads(f.read())
        # trung bình score (điểm chính xác về sự miêu tả đó)
        label_score = np.array([i['score'] for i in jsonContent['labelAnnotations']]).mean()
        # 1 câu nối tất cả description - miêu tả bức ảnh đó
        description_image = ' '.join([i['description'] for i in jsonContent['labelAnnotations']])
        # trung bình score của color (điểm chính xác về sự xuất hiện màu đó)
        color_score = np.array([i['score'] for i in jsonContent['imagePropertiesAnnotation']['dominantColors']['colors']]).mean()
        # trung bình pixelFraction
        pixelFraction = np.array([i['pixelFraction'] for i in jsonContent['imagePropertiesAnnotation']['dominantColors']['colors']]).mean()
        metadate = [label_score,description_image,color_score,pixelFraction]
        print(data['PetID'],end='\r')
    return metadata

* **Vấn đề:** load dữ liệu từ file json Metadata miêu tả ảnh của các vật nuôi
* **Cách làm:** tìm các PetID có file json trong thư mục bằng thư viện os. Load file json thành dict, lấy các trường
    *     description - miêu tả bức ảnh đó
    *     trung bình score (điểm chính xác về sự miêu tả đó)
    *     trung bình score của color (điểm chính xác về sự xuất hiện màu đó)
    *     trung bình pixelFraction
*     Những PetID không có ảnh thì để [0,'',0,0], hoặc thiếu thì để chỗ đấy = 0. Những PetID có nhiều hơn 1 ảnh thì lâý file json của ảnh thứ nhất -1.json

* **Kết quả:**
    * mảng 14993 hàng chứa các thuộc tính trên cho tập train
    * mảng 3971 hàng chứa các thuộc tính trên cho tập test


# Load tất cả dữ liệu

In [ ]:
# load dữ liệu
train = pd.read_csv(r'../input/petfinder-adoption-prediction/train/train.csv')
test = pd.read_csv(r'../input/petfinder-adoption-prediction/test/test.csv')

# load sentiment đã đưọc lấy ra từ file json sentiment và save thành csv
sentiment = pd.read_csv(r'../input/sentiment-petfinder/sentiment.csv')
sentiment_test =  pd.read_csv(r'../input/sentiment-petfinder/sentiment_test.csv')

# load sentimentEntity đã đưọc lấy ra từ file json sentiment và save thành csv
sentimentEntity = pd.read_csv(r'../input/sentimententity-petfinder/sentimentEntity.csv').drop('Unnamed: 0', axis=1)
sentimentEntity_test = pd.read_csv(r'../input/sentimententity-petfinder/sentimentEntity_test.csv')

# load metadata đã đưọc lấy ra từ file json metadata và save thành csv
metadata = pd.read_csv(r'../input/metadata-petfinder/metadata.csv')
metadata_test = pd.read_csv(r'../input/metadata-petfinder/metadata_test.csv')

* **Vấn đề:** Load các file json khá lâu
* **Cách làm:** Load file json rồi save thành file csv đẩy lên dataset rồi load ra mỗi khi cần
* **Kết quả:** Có các file đã được load sẵn thành csv tương ứng mỗi file là 1 dòng

In [ ]:
train = pd.concat([train,sentiment,sentimentEntity,metadata], axis=1)
test = pd.concat([test,sentiment_test,sentimentEntity_test,metadata_test], axis=1)

In [ ]:
def preProcesser(data):
    # Thêm 1 trường con vật đấy có thuẩn chủng hay không nếu 1 trong 2 trường Breed = 0 hoặc 2 trường Breed bằng nhau
    data['IsPureBreed'] = (data["Breed1"] == 0) | (data["Breed2"] == 0) | (data["Breed1"] == data["Breed2"])
    
    # Thêm 1 trưòng con vật đấy có ảnh hay không, có nếu PhotoAmt > 0
    data['HavePhoto'] = data['PhotoAmt']
    data.loc[data['HavePhoto']==0, 'HavePhoto'] = False
    data.loc[data['HavePhoto']!=0, 'HavePhoto'] = True
    # Thêm 1 trường chuẩn hoá trường ảnh bằng cách chia cho số lượng ảnh lớn nhất
    data['PhotoAmt_Normalize'] = data['PhotoAmt']/data['PhotoAmt'].max()
    
    # Thêm 1 trường độ dài của đoạn văn miêu tả bằng cách tìm độ dài của trưòng Description sau đó chuẩn hoá chia cho dữ liệu có độ dài lớn nhất
    data['DescriptionLength'] = data['Description'].map(str).apply(len)
    data['DescriptionLength'] = data['DescriptionLength']/data['DescriptionLength'].max()
    
    # Thêm 1 trưòng gán nhãn cho RescuerID, chuẩn hoá lại từ 0 -> hơn 5000
    le = preprocessing.LabelEncoder()
    le.fit(data['RescuerID'])
    data['EncodeRescuerID'] = le.transform(data['RescuerID'])
    # Thêm 1 trường đếm số lượng con vật mà từng RescuerID có
    df = data.groupby(['RescuerID'])['PetID'].count().reset_index()
    df['RescuerIDCount'] = df['PetID']
    df = df.drop('PetID', axis=1)
    data = pd.merge(data,df, on="RescuerID")
    
    # Thêm 1 trường số lượng màu sắc của con vật đấy nếu 2 color = 0 thì con vật đó 1 màu,...
    temp = ((data['Color1']==0) & (data['Color2']==0))|((data['Color1']==0) & (data['Color3']==0))|((data['Color3']==0) & (data['Color2']==0))
    data['NumberColor'] = [1 if (tmp == True) else 0 for tmp in temp]
    temp = ((data['Color1']!=0) & (data['Color2']!=0) & (data['Color3']==0))|((data['Color1']!=0) & (data['Color3']!=0)& (data['Color2']==0))|((data['Color3']!=0) & (data['Color2']!=0) & (data['Color1']==0))
    data.loc[[tmp == True for tmp in temp], 'NumberColor'] = 2
    temp = ((data['Color1']!=0) & (data['Color2']!=0) & (data['Color3']!=0))
    data.loc[[tmp == True for tmp in temp], 'NumberColor'] = 3
    
    # Thêm 1 trường tính tuổi theo năm, vì tuổi ở đấy đang tính theo tháng nên chia cho 12
    data['AgeYear'] = data['Age']//12
    
    # Thêm 1 trường xem con vật đấy có phải là con vật duy nhất trong 1 bài post không
    data['onePet'] = (data['Quantity']==1)
    
    # Thêm 1 truờng con vật đấy có video không
    data['HaveVideo'] = (data['VideoAmt']!=0)
    
    # Thêm 1 trưòng sentiment bằng cách nhân mức độ tích cực với mức độ cảm xúc với nhau
    data['Sentiment'] = data.apply(sentimet, axis=1)
    
    # Thêm 1 trường sentiment bằng cách đặt threshold 4 mức độ Positive, Negative, Neutral, Mixed
    data['SentimentThreshold'] = data.apply(sentimentThreshold, axis=1)
    
    return data

def sentimet(data):
    # Hàm trả về tích mức độ tích cực Score với mức độ cảm xúc Magnitude
    return data['SentimentScore']*data['SentimentMagnitude']
def sentimentThreshold(data):
    threshold1 = 0.6
    threshold2 = - threshold1
    thOfMagnitude = 3
    Positive = 5
    Negative = -5
    Neutral = 0
    Mixed = 2
    temp = 0
    # Positive nếu score >= 0.6
    if data['SentimentScore'] >= threshold1:
        temp = Positive
    # Mixed nếu -0.6<score<0.6 và magnitude >= 3
    elif data['SentimentScore'] > threshold2 and data['SentimentScore'] < threshold1 and data['SentimentMagnitude'] >= thOfMagnitude:
        temp = Mixed
    # Neutral nếu -0.6<score<0.6 và magnitude < 3
    elif data['SentimentScore'] > threshold2 and data['SentimentScore'] < threshold1 and data['SentimentMagnitude'] < thOfMagnitude:
        temp = Neutral
    # Negative nếu score <= -0.6
    elif data['SentimentScore'] <= threshold2:
        temp = Negative
    # Trả về kết quả ứng với từng mức độ Positive=5, Negative=-5, Neutral=0, Mixed=2
    return temp

* **Vấn đề:** Cần mở rộng thêm tập dữ liệu vì có khá ít feature

* **Cách làm:** Thêm các trường mới:
    * **IsPureBreed:** trường con vật đấy có thuẩn chủng hay không nếu 1 trong 2 trường Breed = 0 hoặc 2 trường Breed bằng nhau
    * **HavePhoto:** trưòng con vật đấy có ảnh hay không, có nếu PhotoAmt > 0
    * **PhotoAmt_Normalize:** trường chuẩn hoá trường ảnh bằng cách chia cho số lượng ảnh lớn nhất
    * **DescriptionLength:** trường độ dài của đoạn văn miêu tả bằng cách tìm độ dài của trưòng Description sau đó chuẩn hoá chia cho dữ liệu có độ dài lớn nhất
    * **EncodeRescuerID:** trưòng gán nhãn cho RescuerID, chuẩn hoá lại từ 0 -> hơn 5000
    * **RescuerIDCount:** trường đếm số lượng con vật mà từng RescuerID có
    * **NumberColor:** trường số lượng màu sắc của con vật đấy nếu 2 color = 0 thì con vật đó 1 màu,...
    * **AgeYear:** trường tính tuổi theo năm, vì tuổi ở đấy đang tính theo tháng nên chia cho 12
    * **onePet:** trường xem con vật đấy có phải là con vật duy nhất trong 1 bài post không
    * **HaveVideo:** truờng con vật đấy có video không
    * **Sentiment:** trưòng sentiment bằng cách nhân mức độ tích cực với mức độ cảm xúc với nhau
    * **SentimentThreshold:** trường sentiment bằng cách đặt threshold 4 mức độ Positive, Negative, Neutral, Mixed
    
    
* **Kết quả:** Ta có thêm 12 trường dữ liệu nữa

In [ ]:
train_pre = preProcesser(train)
test_pre = preProcesser(test)

In [ ]:
def preProcesserText(data):
    # Chuyển dữ liệu đang ở dạng object sang unicode
    a = data['Description'].values.astype('U')
    # Sử dụng regex để loại bỏ tất cả những gì không phải chữ cái tiếng anh, số, dấu ' và - ra (bao gồm cả từ tiếng Trung)
    for i in range(len(a)):
        a[i] = re.sub('[^a-zA-Z0-9\'\-]', ' ', a[i])
    # Áp dụng tf-idf với stopwords là English để trả về 1 tập dữ liệu được chuẩn hoá dựa vào (tf) tần số xuất hiện, (idf) giảm giá trị những từ xuất hiện ở nhiều văn bản
    vectorizer = TfidfVectorizer(stop_words = set(stopwords.words('english')))
    X = vectorizer.fit_transform(a)
    # Giảm chiều dữ liệu bằng SVD thành 8 trường
    svd = TruncatedSVD(n_components=8, random_state=31)
    svd_col = pd.DataFrame(svd.fit_transform(X))
    svd_col = svd_col.add_prefix('DescriptionSVD')
    data = pd.concat([data,svd_col], axis=1)
    
    #Chuyển dữ liệu đang ở dạng object sang unicode
    a = data['SentimentEntity'].values.astype('U')
    # Sử dụng regex để loại bỏ tất cả những gì không phải chữ cái tiếng anh, số, dấu ' và - ra (bao gồm cả từ tiếng Trung)
    for i in range(len(a)):
        a[i] = re.sub('[^a-zA-Z0-9\'\-]', ' ', a[i])
    # Áp dụng tf-idf với stopwords là English để trả về 1 tập dữ liệu được chuẩn hoá dựa vào (tf) tần số xuất hiện, (idf) giảm giá trị những từ xuất hiện ở nhiều văn bản
    vectorizer = TfidfVectorizer(stop_words = set(stopwords.words('english')))
    X = vectorizer.fit_transform(a)
    # Giảm chiều dữ liệu bằng SVD thành 8 trường
    svd = TruncatedSVD(n_components=8, random_state=31)
    svd_col = pd.DataFrame(svd.fit_transform(X))
    svd_col = svd_col.add_prefix('SentimentEntitynSVD')
    data = pd.concat([data,svd_col], axis=1)
    
    #Chuyển dữ liệu đang ở dạng object sang unicode
    a = data['DescriptionImage'].values.astype('U')
    # Sử dụng regex để loại bỏ tất cả những gì không phải chữ cái tiếng anh, số, dấu ' và - ra (bao gồm cả từ tiếng Trung)
    for i in range(len(a)):
        a[i] = re.sub('[^a-zA-Z0-9\'\-]', ' ', a[i])
    # Áp dụng tf-idf với stopwords là English để trả về 1 tập dữ liệu được chuẩn hoá dựa vào (tf) tần số xuất hiện, (idf) giảm giá trị những từ xuất hiện ở nhiều văn bản
    vectorizer = TfidfVectorizer(stop_words = set(stopwords.words('english')))
    X = vectorizer.fit_transform(a)
    # Giảm chiều dữ liệu bằng SVD thành 8 trường
    svd = TruncatedSVD(n_components=8, random_state=31)
    svd_col = pd.DataFrame(svd.fit_transform(X))
    svd_col = svd_col.add_prefix('DescriptionImageSVD')
    data = pd.concat([data,svd_col], axis=1)
    
    return data

* **Vấn đề:** Ta có 3 trường text Description, SentimentEntity, DescriptionImage và không thể đưa vào học được ngay
* **Cách làm:** 
    * Chuyển dữ liệu đang ở dạng object sang unicode
    * Sử dụng regex để loại bỏ tất cả những gì không phải chữ cái tiếng anh, số, dấu ' và - ra (bao gồm cả từ tiếng Trung)
    * Áp dụng tf-idf với stopwords là English để trả về 1 tập dữ liệu được chuẩn hoá dựa vào (tf) tần số xuất hiện, (idf) giảm giá trị những từ xuất hiện ở nhiều văn bản
    * Giảm chiều dữ liệu bằng SVD thành 8 trường
    * Concat vào cuổi bảng dữ liệu
* **Kết quả:** Ta có thêm 24 trưòng dữ liệu text được chuẩn hoá

In [ ]:
train_text = preProcesserText(train_pre)
test_text = preProcesserText(test_pre)

# Load ảnh

In [ ]:
inp = Input((256,256,3))
#Sử dụng pretrain model với 2 model đã sử dụng qua là VGG19 và DenseNet121 (include_top=False lấy phần feature phía duới, bỏ phần Classifier tầng phía trên) để trích xuất đặc trưng của từng bức ảnh
model = DenseNet121(input_tensor = inp, weights='imagenet',include_top=False)
x = model.output
# Output của bưóc trên là 1x7x7x256, ta cho qua 1 lần GlobalAveragePooling2D để kích thưóc còn 1x1024 và AveragePooling để có kích thưóc nhỏ hơn là 1x256
x = GlobalAveragePooling2D()(x)
x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
x = AveragePooling1D(4)(x)
out = Lambda(lambda x: x[:,:,0])(x)
model = Model(inp,out)

def append(img):
    # Lấy đường dẫn của ảnh
    if img['HavePhoto']:
        return '../input/petfinder-adoption-prediction/train_images/'+img['PetID']+'-1.jpg'
    return ''

def append_test(img):
    # Lấy đường dẫn của ảnh
    if img['HavePhoto']:
        return '../input/petfinder-adoption-prediction/test_images/'+img['PetID']+'-1.jpg'
    return ''

def loadImage(data):
    if data['Input_image']!='':
        # Load bức ảnh và resize kích thưóc về 256x256x3
        img = image.load_img(data['Input_image'], target_size=(256, 256))
        # chuyển buớc ảnh về ma trận numpy array
        x = image.img_to_array(img)
        # thêm 1 trưòng nữa để đưa bức ảnh thành 1x256x256x3 trước khi đưa vào model
        x = np.expand_dims(x, axis=0)
        # Sử dụng pretrain model với 2 model đã sử dụng qua là VGG19 và DenseNet121 (include_top=False lấy phần feature phía duới, bỏ phần Classifier tầng phía trên) để trích xuất đặc trưng của từng bức ảnh
        a = model.predict(x)
        print(data['PetID'], end='\r')
        return a[0]
    return np.zeros(256)

train_image = train.copy()
# Lấy đường dẫn của ảnh
train_image['Input_image'] = train_image.apply(append, axis=1)
# Trích xuất đặc trưng
train_image['Image'] = train_image.apply(loadImage, axis=1)
image = pd.DataFrame(train_image['Image'].to_list(), columns=[i for i in range(256)])
# giảm chiều dữ liệu bằng SVD (hoặc NMF) còn 32 trường
svd=TruncatedSVD(n_components=32)
image=pd.DataFrame(svd.fit_transform(image))
image = image.add_prefix('imageSVD')
# image = image.add_prefix('imageNMF')

test_image = test.copy()
# Lấy đường dẫn của ảnh
test_image['Input_image'] = test_image.apply(append_test, axis=1)
# Trích xuất đặc trưng
test_image['Image'] = test_image.apply(loadImage, axis=1)
image_test = pd.DataFrame(test_image['Image'].to_list(), columns=[i for i in range(256)])
# giảm chiều dữ liệu bằng SVD (hoặc NMF) còn 32 trường
svd=TruncatedSVD(n_components=32)
image_test=pd.DataFrame(svd.fit_transform(image_test))
image_test = image_test.add_prefix('imageSVD')
# image_test = image_test.add_prefix('imageNMF')

# image.to_csv('imageNMFDenseNet121.csv', index=False)
# image_test.to_csv('imageNMFDenseNet121_test.csv', index=False)
# image.to_csv('imageSVDDenseNet121.csv', index=False)
# image_test.to_csv('imageSVDDenseNet121_test.csv', index=False)

* **Vấn đề:** Ta có 1 tập dữ liệu ảnh có nhiều kích thưóc khác nhau và khá lớn, cần chuẩn hoá về 1 số đặc trưng nhất định
* **Cách làm:** 
    * Lấy đường dẫn của ảnh
    * Load bức ảnh và resize kích thưóc về 256x256x3, chuyển buớc ảnh về ma trận numpy array, thêm 1 trưòng nữa để đưa bức ảnh thành 1x256x256x3 trước khi đưa vào model
    * Sử dụng pretrain model với 2 model đã sử dụng qua là VGG19 và DenseNet121 (include_top=False lấy phần feature phía duới, bỏ phần Classifier tầng phía trên) để trích xuất đặc trưng của từng bức ảnh
    * Output của bưóc trên là 1x7x7x256, ta cho qua 1 lần GlobalAveragePooling2D để kích thưóc còn 1x1024 và AveragePooling để có kích thưóc nhỏ hơn là 1x256
    * Sau đó ta lại giảm chiều dữ liệu bằng SVD (hoặc NMF) còn 32 trường
* **Kết quả:** Ta có thêm 32 trưòng đặc trưng được lấy ra từ ảnh


In [ ]:
# VGG19_NMF
image = pd.read_csv(r'../input/imagenmf-petfinder/imageNMF.csv')
image_test = pd.read_csv(r'../input/imagenmf-petfinder/imageNMF_test.csv')
# DensenNet121_NMF
imageDenseNet121 = pd.read_csv(r'../input/imagenmf-petfinder/imageNMFDenseNet121.csv')
imageDenseNet121_test = pd.read_csv(r'../input/imagenmf-petfinder/imageNMFDenseNet121_test.csv')
imageDenseNet121 = imageDenseNet121.add_prefix('DenseNet121')
imageDenseNet121_test = imageDenseNet121_test.add_prefix('DenseNet121')
# DensenNet121_SVD
imageDenseNet121SVD = pd.read_csv(r'../input/imagenmf-petfinder/imageSVDDenseNet121.csv')
imageDenseNet121SVD_test = pd.read_csv(r'../input/imagenmf-petfinder/imageSVDDenseNet121_test.csv')
imageDenseNet121SVD = imageDenseNet121SVD.add_prefix('DenseNet121')
imageDenseNet121SVD_test = imageDenseNet121SVD_test.add_prefix('DenseNet121')

* **Vấn đề:** Trích xuất đặc trưng từ ảnh mất rất nhiều thời gian
* **Cách làm:** Save lại vào file csv và load ra dùng mỗi khi cần
* **Kết quả:** file csv và load ra dùng mỗi khi cần

In [ ]:
train_image = pd.concat([train_text,image,imageDenseNet121,imageDenseNet121SVD], axis=1)
test_image = pd.concat([test_text,image_test,imageDenseNet121_test,imageDenseNet121SVD_test], axis=1)

In [ ]:
train_image.to_csv('train_petfinder4.csv', index=False)
test_image.to_csv('test_petfinder4.csv', index=False)